In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b88b54f0b49510a023dc1fe60af39055487e3c6eff88cb2c6f7694ffd0b6489e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import os
import pandas as pd
import numpy as np
import torch
#Libraries for sentence transformer
from sentence_transformers import SentenceTransformer, util, InputExample, models
from tqdm.notebook import tqdm_notebook # for progress bar
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
#Libraries for visualization
import seaborn as sns
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def load_data(path):
    import os
    
    #_____________ Read data from CISI.ALL file and store in dictinary ________________
    
    with open(os.path.join(path, 'CISI.ALL')) as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
 
    doc_set = {}
    doc_id = ""
    doc_text = ""

    for l in lines:
        if l.startswith(".I"):
            doc_id = l.split(" ")[1].strip() 
        elif l.startswith(".X"):
            doc_set[doc_id] = doc_text.lstrip(" ")
            doc_id = ""
            doc_text = ""
        else:
            doc_text += l.strip()[3:] + " " 

    print(f"Number of documents = {len(doc_set)}")
    print(doc_set["1"]) 
    
    
    #_____________ Read data from CISI.QRY file and store in dictinary ________________
    
    with open(os.path.join(path, 'CISI.QRY')) as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
          
    qry_set = {}
    qry_id = ""
    for l in lines:
        if l.startswith(".I"):
            qry_id = l.split(" ")[1].strip() 
        elif l.startswith(".W"):
            qry_set[qry_id] = l.strip()[3:]
            qry_id = ""

    print(f"\n\nNumber of queries = {len(qry_set)}")    
    print(qry_set["1"]) 
    
    
    #_____________ Read data from CISI.REL file and store in dictinary ________________
    
    rel_set = {}
    with open(os.path.join(path, 'CISI.REL')) as f:
        for l in f.readlines():
            qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
            doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

            if qry_id in rel_set:
                rel_set[qry_id].append(doc_id)
            else:
                rel_set[qry_id] = []
                rel_set[qry_id].append(doc_id)

    print(f"\n\nNumber of mappings = {len(rel_set)}")
    print(rel_set["1"]) 
    
    doc_set = {int(id):doc for (id,doc) in doc_set.items()}
    qry_set = {int(id):qry for (id,qry) in qry_set.items()}
    rel_set = {int(qid):list(map(int, did_lst)) for (qid,did_lst) in rel_set.items()}
    
    return doc_set, qry_set, rel_set

In [182]:
# Recall@K = TP/(TP+FN)
def recall_k(ground_truth, predictions, k):
  avg_recall = 0
  for query_id in ground_truth:    
    truth_set = set(ground_truth[query_id])
    pred_set = set(predictions[query_id][:k])
    result = round(len(truth_set & pred_set) / float(len(truth_set)), 2) 
    avg_recall += result
    avg_recall /= len(ground_truth)

  return round(avg_recall, 3)

In [183]:
# Precision@K = TP/(TP+FP)
def precision_k(ground_truth, predictions, k):
  avg_precision = 0
  for query_id in ground_truth:    
    truth_set = set(ground_truth[query_id]) 
    pred_set = set(predictions[query_id][:k])
    result = round(len(truth_set & pred_set) / float(len(pred_set)), 2) 
    avg_precision += result
  avg_precision /= len(ground_truth)

  return round(avg_precision, 3)

In [184]:
def get_first_relevent_docid(predictions, truth):
    for doc_id in predictions:
        is_exist = doc_id in truth 
        if is_exist:
            return predictions.index(doc_id)+1 
    else:
        return -1


def calc_mrr(doc_scores, rel_set):
    Q = len(rel_set) 
    cumulative_reciprocal = 0  
    
    for query_id in rel_set:
        first_result = get_first_relevent_docid(doc_scores[query_id], rel_set[query_id])
        first_result_rank = len(doc_scores[1])+1 if first_result<1 else first_result 
        reciprocal = 1 / first_result_rank
        cumulative_reciprocal += reciprocal
        
    mrr = 1/Q * cumulative_reciprocal 
    return round(mrr,3)


In [185]:
def map_k(rel_set, doc_scores, K):
    Q = len(rel_set) 
    avg_precision = [] 

    for query_id in rel_set:
        precision_relevance_summation = 0

        for k in range(0,K):
            # calculate precision@k
            truth_set = set(rel_set[query_id])
            pred_set = set(doc_scores[query_id][:k+1])
            precision_at_k = round(len(truth_set & pred_set) / float(len(pred_set)), 2) 
            
            rel_k = 1 if doc_scores[query_id][k] in rel_set[query_id] else 0 
            precision_relevance_summation += precision_at_k * rel_k 
            
        avg_precision_q = precision_relevance_summation / len(rel_set[query_id])
        avg_precision.append(avg_precision_q)

    map_k = sum(avg_precision) / Q 
    return round(map_k, 3)

In [6]:
doc_set, qry_set, rel_set = load_data('/kaggle/input/all-data')

Number of documents = 1460
18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


Number of queries = 112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


Number of mappings = 76
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '1

In [7]:
#Split data into 70-30 
qry_set_train_split = int(len(qry_set)*70/100)
doc_set_train_split = int(len(doc_set)*70/100 )
print(f"Train split\n qry length: {qry_set_train_split}, doc length: {doc_set_train_split}\n")

qry_set_valid_split = int(len(qry_set)-qry_set_train_split)
doc_set_valid_split = int(len(doc_set)-doc_set_train_split)
print(f"Train split\n qry length: {qry_set_valid_split}, doc length: {doc_set_valid_split}")

Train split
 qry length: 78, doc length: 1022

Train split
 qry length: 34, doc length: 438


In [9]:
def is_similar(query_id, doc_id):
      return 0.9 if query_id in rel_set.keys() and doc_id in rel_set[query_id] else 0.1

In [12]:
#Reformat the input data for Finetuning Sentence Transformer using MNR Loss
train_samples = []

for query_id in tqdm_notebook(range(qry_set_train_split)):
  for doc_id in range(doc_set_train_split):
    train_samples.append(
        InputExample(
            texts=[qry_set[query_id+1], doc_set[doc_id+1]],
            label= is_similar(query_id+1, doc_id+1)
            ))

  0%|          | 0/78 [00:00<?, ?it/s]

In [14]:
#Reformat the input data for Finetuning Sentence Transformer using MNR Loss
validation_samples = []

for query_id in tqdm_notebook(range(qry_set_train_split, len(qry_set))):
  for doc_id in range(doc_set_train_split, len(doc_set)):
    validation_samples.append(
        InputExample(
            texts=[qry_set[query_id+1], doc_set[doc_id+1]],
            label= is_similar(query_id+1, doc_id+1)
            ))

  0%|          | 0/34 [00:00<?, ?it/s]

In [15]:
#Define Model, Dataloader, loss, evaluator
model = SentenceTransformer('all-MiniLM-L6-v2')

loader = DataLoader(train_samples, shuffle=True, batch_size=16)

loss = losses.CosineSimilarityLoss(model)

In [16]:
#train for a single epoch and warmup for the first 10% of our training steps.
model.fit(
    train_objectives=[(loader, loss)],
    epochs=1,
    warmup_steps = 100
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4983 [00:00<?, ?it/s]

In [186]:
#compute the embeddings for the corpus
docs_embeddings = model.encode(list(doc_set.values()), convert_to_tensor=True)
docs_embeddings = np.asarray(docs_embeddings.cpu()).astype('float32')

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [187]:
docs_embeddings.shape

(1460, 384)

## Bi-encoder only

In [188]:
import faiss

In [189]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(len(docs_embeddings[0])))
faiss.normalize_L2(docs_embeddings)
index.add_with_ids(docs_embeddings,np.array(range(0,len(docs_embeddings))).astype(np.int64))
faiss.write_index(index,'pip_for_val')
index = faiss.read_index('pip_for_val')

In [190]:
predicted_doc_scores = {}

for i in range(len(qry_set)):
    query_vector = model.encode([qry_set[i+1]])
    faiss.normalize_L2(query_vector)
    top_k = index.search(query_vector,k=10)
    predicted_doc_scores[i+1] = [x+1 for x in top_k[1].tolist()[0]]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [191]:
print(f"Recall@10 = {recall_k(rel_set, predicted_doc_scores, 10)}") 

Recall@10 = 0.007


In [192]:
print(f"Precision@10 = {precision_k(rel_set, predicted_doc_scores, 10)}")

Precision@10 = 0.593


In [194]:
mrr = calc_mrr(predicted_doc_scores, rel_set)
print(f"Mean Reciprocal Rank (MRR): {mrr}")

Mean Reciprocal Rank (MRR): 0.83


In [195]:
map_10 = map_k(rel_set, predicted_doc_scores, K=10)
print(f"MAP@10 = {map_10}")

MAP@10 = 0.192


## bi + cross encoder

In [196]:
predicted_doc_scores = {}

for i in range(len(qry_set)):
    query_vector = model.encode([qry_set[i+1]])
    faiss.normalize_L2(query_vector)
    top_k = index.search(query_vector,k=10)
    predicted_doc_scores[i+1] = [x+1 for x in top_k[1].tolist()[0]]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [197]:
from sentence_transformers.cross_encoder import CrossEncoder

In [198]:
model1 = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [199]:
predicted_score_all = []

for ind,q in enumerate(predicted_doc_scores):
    
    predicted_score = []
    doc = predicted_doc_scores[q]
    for d in doc:
        predicted_score.append(model1.predict([qry_set[q] , doc_set[d]]))
    predicted_score_all.append(predicted_score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [200]:
final_reranked_doc = []
for i in range(len(predicted_score_all)):
    df = pd.concat([pd.DataFrame(predicted_doc_scores[i+1]),pd.DataFrame(predicted_score_all[i])],axis=1)
    df.columns = ['doc_id','score']
    df = df.sort_values('score',ascending=False)
    final_reranked_doc.append(list(df.iloc[:10]['doc_id']))

In [202]:
predicted_doc_scores = {}

for i, l in enumerate(final_reranked_doc):
    predicted_doc_scores[i+1] = l


In [203]:
print(f"Recall@10 = {recall_k(rel_set, predicted_doc_scores, 10)}") 

Recall@10 = 0.007


In [204]:
print(f"Precision@10 = {precision_k(rel_set, predicted_doc_scores, 10)}")

Precision@10 = 0.593


In [205]:
mrr = calc_mrr(predicted_doc_scores, rel_set)
print(f"Mean Reciprocal Rank (MRR): {mrr}")

Mean Reciprocal Rank (MRR): 0.805


In [206]:
map_10 = map_k(rel_set, predicted_doc_scores, K=10)
print(f"MAP@10 = {map_10}")

MAP@10 = 0.175
